# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import *

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [2]:
sc
sqlContext = SQLContext(sc)

### Student Section - Please compete all the functions below

#### Function to return GT Username

In [3]:
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'yyu441'

#### Function to load data

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP storage bucket#
    #                                                              #        
    ################################################################
    # 'gs://yyu441/yellow_tripdata_2019-01.csv'
    spark = SparkSession.builder.appName('GCSFilesRead').getOrCreate()
    df = spark.read.csv(gcp_storage_path, header=True)
    df = df.select(col('VendorID'),
                   col('tpep_pickup_datetime'),
                   col('tpep_dropoff_datetime'),
                   col('passenger_count'),
                   col('trip_distance').cast('Decimal').alias('trip_distance'),
                   col('RatecodeID'),
                   col('store_and_fwd_flag'),
                   col('PULocationID'),
                   col('DOLocationID'),
                   col('payment_type'),
                   col('fare_amount').cast('Decimal').alias('fare_amount'),
                   col('extra'),
                   col('mta_tax'),
                   col('tip_amount').cast('Decimal').alias('tip_amount'),
                   col('tolls_amount').cast('Decimal').alias('tolls_amount'),
                   col('improvement_surcharge'),
                   col('total_amount'))
    return df

In [57]:
# df = load_data('gs://yyu441/yellow_tripdata_2019-01.csv')
# df.show(2)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|            2|         1|                 N|         151|         239|           1|          7|  0.5|    0.5|         2|           0|                  0.3|        9.95|
|       1| 2019-01-01 00:59:47|  2019-01-01 01:1

In [61]:
# df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: decimal(10,0) (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: decimal(10,0) (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: decimal(10,0) (nullable = true)
 |-- tolls_amount: decimal(10,0) (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



#### Function to exclude trips that don't have a pickup location

In [5]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no pickup locations               #
    # Note: Exclude  nulls, blanks and zeros                       #        
    ################################################################
    # https://stackoverflow.com/questions/44163153/how-to-drop-rows-with-nulls-in-one-column-pyspark
    df = df.filter(~(col('PULocationID').isNull() | isnan(col('PULocationID')) | (trim(col('PULocationID')) == "")))
    return df

In [91]:
# df.filter(~(col('PULocationID').isNull() | isnan(col('PULocationID')) | (trim(col('PULocationID')) == ""))).count()

7667792

In [89]:
# t = spark.createDataFrame([
#     ("", 1, 2.0), 
#     ("foo", None, 3.0),
#     ("bar", 1, float("NaN")), 
#     ("good", 42, 42.0)])
# t.filter(~(col('_1').isNull() | isnan(col('_1')) | (trim(col('_1')) == ""))).show()

+----+----+----+
|  _1|  _2|  _3|
+----+----+----+
| foo|null| 3.0|
| bar|   1| NaN|
|good|  42|42.0|
+----+----+----+



#### Function to exclude trips with no distance

In [6]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no trip distances                 #
    # Note: Exclude  nulls, blanks and zero                        #        
    ################################################################
    df = df.filter(~(col('trip_distance').isNull() | isnan(col('trip_distance')) | (trim(col('trip_distance')) == "")))
    return df

#### Function to include fare amount between the range of 20 to 60 Dollars

In [15]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars (including 20 and 60 dollars)               #        
    ################################################################
    df = df.filter((col('fare_amount') >= 20) & (col('fare_amount') <= 60))
    return df

#### Function to get the highest tip amount

In [16]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get the highest tip                                  #
    #                                                              #        
    ################################################################
    max_tip = df.withColumn('tip_amount', round('tip_amount', 2)).orderBy(col('tip_amount').desc()).limit(1).collect()[0]
    return max_tip

#### Function to get total toll amount

In [17]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    total_toll = df.withColumn('tolls_amount', round('tolls_amount', 2)).groupBy().sum().collect()[0]
    return total_toll

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [18]:
gcp_storage_path = "gs://yyu441/yellow_tripdata_2019-01.csv"
df = load_data(gcp_storage_path)
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: decimal(10,0) (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: decimal(10,0) (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: decimal(10,0) (nullable = true)
 |-- tolls_amount: decimal(10,0) (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



#### Print total numbers of rows in the dataframe

In [19]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [20]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

7667792

#### Print total number of rows in the dataframe after exclude trips with no distance

In [21]:
df_no_trip_distance = exclude_no_tripdistance(df)
df_no_trip_distance.count()

7667792

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [22]:
df_include_fare_range = include_fare_range(df)
df_include_fare_range.count()

1011593

#### Print the highest tip amount

In [23]:
max_tip = get_highest_tip(df)
print(max_tip)

Row(VendorID='4', tpep_pickup_datetime='2019-01-05 23:19:48', tpep_dropoff_datetime='2019-01-05 23:25:06', passenger_count='1', trip_distance=Decimal('1'), RatecodeID='1', store_and_fwd_flag='N', PULocationID='239', DOLocationID='24', payment_type='1', fare_amount=Decimal('7'), extra='0.5', mta_tax='0.5', tip_amount=Decimal('787'), tolls_amount=Decimal('0'), improvement_surcharge='0.3', total_amount='795.05')


#### Print the total toll amount

In [24]:
total_toll = get_total_toll(df)
print(total_toll)

Row(sum(trip_distance)=Decimal('21753865'), sum(fare_amount)=Decimal('97013077'), sum(tip_amount)=Decimal('14047953'), sum(tolls_amount)=Decimal('2525415'))
